# Introducción a la Creación de Chatbots con LangChain y Modelos de Código Abierto

Por William Wong Garay - CTO de Azist.ai

### Herramientas que usaremos:

- [LangChain](https://python.langchain.com/): Framework para desarrollar aplicaciones poteniadas por grandes modelos de lenguaje natural (LLMs por sus siglas en inglés).
- [Groq](https://groq.com/):  Es una startup de Silicon Valley que provee servicios en la nube para acceder a LLMs OpenSource con respuestas de alta velocidad, gracias a sus chips LPU especializados. 
- Otras herramientas: Usaremos otras librerías como BeautifulSoup para la carga de documentos via web, y Chroma para la búsqueda y recuperación de información.

In [1]:
%pip install --upgrade pip -q
%pip install langchain -q
%pip install python-dotenv -q
%pip install langchain-groq -q
%pip install langchain-community -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Inicialización del LLM para el modelo de Chat

Para el ejemplo usaremos el modelo Llama 3.1 mediante la API de Groq.

Los modelos tipo Chat son modelos de lenguaje natural que han sido entrenados para recibir como entrada una pregunta de un user (HumanMessage) y devolver una respuesta de un agente virtual (AIMessage)

Pueden ver una lista completa de las librerías de chat disponibles en https://python.langchain.com/v0.1/docs/integrations/chat/

In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv(override=True) # Carga el GROQ_API_KEY desde el archivo .env

True

In [3]:
llm = ChatGroq(
    temperature=1.0, # Entre 0 y 2. Temperaturas altas generan respuestas más creativas, temperaturas bajas generan respuestas más enfocadas y centradas.
    model="llama-3.1-8b-instant", # Modelo de lenguaje a utilizar. https://console.groq.com/docs/models
    # model="gemma2-9b-it",
    # model="llama3-70b-8192"
)

llm.invoke("Hola, ¿que LLM eres y quien es tu creador?")


AIMessage(content='Hola. Soy un modelo de lenguaje de inteligencia artificial (LLM) desarrollado por Meta AI. Mi nombre es Llama 3.4, aunque mi nombre puede variar dependiendo de la versión o la implementación en la que se encuentre.', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 50, 'total_tokens': 106, 'completion_time': 0.074666667, 'prompt_time': 0.016812282, 'queue_time': 0.0025065059999999986, 'total_time': 0.091478949}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-242ae8e6-b21c-4009-8bac-47a81f1ac639-0', usage_metadata={'input_tokens': 50, 'output_tokens': 56, 'total_tokens': 106})

In [4]:
from langchain_core.messages import HumanMessage

llm.invoke(
    [ HumanMessage(content="Hola, conoces la ciudad de Piura?") ]
)

AIMessage(content='Sí, conozco Piura. Es una ciudad ubicada en el norte del Perú, en la región de Piura. Es famosa por su belleza natural, su rica cultura y su calor tropical. La ciudad se encuentra en la Depresión Piura, junto al río Piura, y es la capital de la región.\n\nPiura es conocida por sus playas de cala, sus pueblos coloniales, sus ruinas arqueológicas y sus paisajes naturales impresionantes. La ciudad cuenta con un rico patrimonio cultural, que refleja la influencia de la cultura inca, la colonia y el mestizaje.\n\nAlgunos sitios turísticos populares en Piura incluyen:\n\n* La plaza de Armas, el corazón histórico de la ciudad, rodeada por edificios coloniales y monumentos importantes.\n* El castillo de la Misericordia, un palacio del siglo XVII con una estupenda arquitectura.\n* La Catedral de Piura, una impresionante estructura gótica del siglo XVI.\n* El Museo Regional de Piura, que cuenta con una colección de arte y artefactos históricos importantes.\n\nLa región de Piur

La conversacion con un Chat puede recibir principalmente 3 tipos de mensajes:

- SystemMessage: Mensajes del sistema, que corresponden al Prompt inicial del Chat. Sirve para darle contexto a la conversación.
- HumanMessage: Mensajes del usuario, que corresponden a las preguntas que el usuario le hace al Chat.
- AIMessage: Mensajes del Chat, que corresponden a las respuestas que el Chat le da al usuario a través del modelo de lenguaje natural.

In [5]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

llm.invoke([
    SystemMessage(content="Eres un asistente virtual. Tu nombre es Azzy y trabajas en Shalom. Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo"),
    HumanMessage(content="Hola, ¿como estas?"),
    AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
    HumanMessage(content="Me das tu nombre? Hacen envios a Trujillo?"),
])


AIMessage(content='Me llamo Azzy. Lamento decir que en Shalom, la empresa donde trabajo, solo hacemos envíos a Piura, Lima y Chiclayo. No ofertamos servicios a Trujillo. ¿Crees que necesitas algún servicio de envío a una de estas ciudades?', response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 124, 'total_tokens': 188, 'completion_time': 0.085333333, 'prompt_time': 0.025154318, 'queue_time': 0.0023518390000000014, 'total_time': 0.110487651}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a62c169-7ff6-4b0c-a6c6-fcdbf1612e7b-0', usage_metadata={'input_tokens': 124, 'output_tokens': 64, 'total_tokens': 188})

El Prompt es una parte importante del Chatbot, ya que es el mensaje inicial que se le da al modelo para que pueda responder de manera coherente.

LangChain nos pemite darle formato al Prompt mediante el uso de templates y variables, la cuales se pueden reemplazar por valores específicos en tiempo de ejecución.

Con el ChatPromptTemplate podremos pre formatear el Prompt del Chatbot y la cadena de mensajes que se le pasará al modelo por medio de MessagePlaceholder.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente virtual. Tu nombre es {assistant_name} y trabajas en {company}. {description}."),
    MessagesPlaceholder(variable_name="messages"),
])

prompt.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es una empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": [
        HumanMessage(content="Hola, ¿como estas?"),
        AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
        HumanMessage(content="Me das tu nombre? Hacen envios a Trujillo?"),
    ]
})

ChatPromptValue(messages=[SystemMessage(content='Eres un asistente virtual. Tu nombre es Azzy y trabajas en Shalom. Shalom es una empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo.'), HumanMessage(content='Hola, ¿como estas?'), AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'), HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?')])

Las cadenas en LangChain permiten combinar componentes del framework para encadenar las salidas de un componente a la entrada de otro.

En este ejemplo, usaremos el ChatPromptTemplate para formatear el Prompt del Chatbot y el MessagePlaceholder para formatear la cadena de mensajes que se le pasará al modelo ChatGroq.

Al hecer el invoke a la cadena, se le debe pasar todas las variables necesarias para que el Chatbot pueda responder de manera correcta.

In [7]:
chain = prompt | llm

chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": [
        HumanMessage(content="Hola, ¿como estas?"),
        AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
        HumanMessage(content="Me das tu nombre? Hacen envios a Trujillo?"),
    ]
})

AIMessage(content='Me llamo Azzy y soy un asistente virtual de la empresa Shalom. Lo siento, pero no realizamos envíos a Trujillo. Nuestra zona de servicio es específica para Piura, Lima y Chiclayo. ¿Podrías necesitar envíos a alguna de estas ciudades?', response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 125, 'total_tokens': 194, 'completion_time': 0.092, 'prompt_time': 0.031763099, 'queue_time': 0.0020335209999999965, 'total_time': 0.123763099}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-c2babfcb-7faa-4333-8f48-c97582ad0563-0', usage_metadata={'input_tokens': 125, 'output_tokens': 69, 'total_tokens': 194})

### Usando un Historial de Conversación

LangChain nos brinda diferentes librerías para trabajar con historiales de conversación, como ChatMessageHistory via langchain memory que permite almacenar en memoria los mensajes de la conversación.

Existen otras librerías que hacen uso de bases de datos permanentes para almacenar los mensajes de la conversación, como PostgresChatMessageHistory que permite almacenar los mensajes de la conversación en una base de datos Postgres.

Pueden ver una lista completa en https://python.langchain.com/v0.1/docs/integrations/memory/

In [8]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()

chat_history.add_user_message("Hola, ¿como estas?")
chat_history.add_ai_message("Estoy bien, gracias por preguntar. En que te puedo ayudar?")
chat_history.add_user_message("Me das tu nombre? Hacen envios a Trujillo?")

chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?')]

In [9]:
chat_history.add_ai_message("Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo")
chat_history.add_user_message("Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda")

chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?'),
 AIMessage(content='Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo'),
 HumanMessage(content='Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda')]

In [10]:
ai_response = chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": chat_history.messages
})

ai_response

AIMessage(content='No hay problema. Me alegra haber podido ayudarte a confirmar que no podemos realizar envíos a Trujillo. Si necesitas ayuda con algo más relacionado con Shalom o tienes alguna pregunta sobre nuestros servicios, no dudes en preguntar. Que tengas un buen día.', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 182, 'total_tokens': 244, 'completion_time': 0.082666667, 'prompt_time': 0.0624739, 'queue_time': 0.004427920000000002, 'total_time': 0.145140567}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-0c6ae794-b273-4f28-815f-d71b0d9fce04-0', usage_metadata={'input_tokens': 182, 'output_tokens': 62, 'total_tokens': 244})

In [11]:
chat_history.add_ai_message(ai_response)
chat_history.add_user_message("Oh si, tengo otra pregunta. Será que piensan abrir una sucursal en Trujillo a futuro?")

ai_response = chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": chat_history.messages
})

ai_response

AIMessage(content='Eso sería un gran paso para Shalom. No tengo información específica al respecto, pero puedo sugerirte que te pongas en contacto con nosotros en un futuro y preguntar directamente a nuestros representantes. La expansión de la empresa es un tema que se discute constantemente, pero por ahora, nuestra servicio se enfoca en Piura, Lima y Chiclayo. Si eso cambia, te lo comunicaré lo primero.', response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 277, 'total_tokens': 372, 'completion_time': 0.126666667, 'prompt_time': 0.056755407, 'queue_time': 0.002431968999999999, 'total_time': 0.183422074}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-42862afb-1445-47e2-afd5-041951aba0e2-0', usage_metadata={'input_tokens': 277, 'output_tokens': 95, 'total_tokens': 372})

In [12]:
chat_history.add_ai_message(ai_response)
chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?'),
 AIMessage(content='Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo'),
 HumanMessage(content='Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda'),
 AIMessage(content='No hay problema. Me alegra haber podido ayudarte a confirmar que no podemos realizar envíos a Trujillo. Si necesitas ayuda con algo más relacionado con Shalom o tienes alguna pregunta sobre nuestros servicios, no dudes en preguntar. Que tengas un buen día.', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 182, 'total_tokens': 244, 'completion_time': 0.082666667, 'prompt_time': 0.0624739, 'queue_time': 0.004427920000000002, 'total_time': 0.145140567}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'lo

### Dándole mayor base de conocimiento al Chatbot con RAG (Retrieval Augmented Generation)

Muchas veces necesitaremos agregarle más contexto a la conversación para que el Chatbot pueda responder de manera más precisa. Sin embargo los LLM's suelen tener una capacidad máxima de tokens que pueden procesar.

Una empresa podría tener una base de datos de sus productos, servicios, o información de contacto, que podría ser usada para responder preguntas específicas de los usuarios, pero agregarlas directamente al modelo podría exceder la capacidad de tokens o generar altos costos de uso.

Para estos casos, LangChain nos permite agregar nuestra base de conocimientos a una base de datos vectorial (Vector Stores). Pueden ver la lista completa de librerías en https://python.langchain.com/v0.1/docs/integrations/vectorstores/

Asi mismo, nos permite recuperar los datos relevantes a usar en nuetro Prompt consultando la base de datos vectorial (Retriever). Pueden ver la lista completa de librerías en https://python.langchain.com/v0.1/docs/integrations/retrievers/

Para este ejemplo usaremos BeautifulSoup para la carga de documentos via web, y Chroma para la búsqueda y recuperación de información.

In [13]:
%pip install --upgrade --quiet langchain-chroma beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


La comunidad de LangChain es muy activa y siempre está creando nuevas librerías y herramientas para acelerar el desarrollo de nuestras aplicaciones. 

Para el ejemplo usaremos WebBaseLoder para cargar documentos via web usando BeautifulSoup.

Pueden ver la lista de Document Loaders disponibles en https://python.langchain.com/v0.1/docs/integrations/document_loaders/

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
    "https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html",
])

data = loader.load()
data

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='\n\n\n\n\nPreguntas Frecuentes - Shalom\n\n\n\n\nBienvenido(a) a Preguntas Frecuentes\n\n\nResuelve tus dudas sobre nuestro servicio de envío.\nTus envíos\n¿Qué tipos de servicios ofrece Shalom?\nEn Shalom, nos especializamos en ofrecer servicios de envío de carga y encomiendas a nivel nacional, con más de 250 destinos terrestres y 15 destinos aéreos. Además, facilitamos el recojo y reparto de tus envíos directamente en tu domicilio. Gracias a nuestras alianzas estratégicas, también podrás realizar tus envíos internacionales de manera eficiente y confiable. Para conocer más visita nuestra página web en el siguiente enlace: https://shalom.com.pe\n¿Cuál es su cobertura?\nTenemos una amplia cobertura en todo el Perú, con más de 211 agencias a

Muchas veces los documentos puede ser muy extensos y dificiles de almacenar en el Vector Store. Además, es una buena práctica almacenar la información por partes semánticamente relacionadas para facilitar la recuperación de la información.

LangChain nos permite partir los documentos con diferentes técnicas, para el ejemplo usaremos RecursiveCharacterTextSplitter, el cual parte el texto en párrafos o grupos de caracteres según la longitud del texto.

Para ver una lista completa de los splitters disponibles pueden visitar https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/ 

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

all_splits

[Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='Preguntas Frecuentes - Shalom\n\n\n\n\nBienvenido(a) a Preguntas Frecuentes'),
 Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='Resuelve tus dudas sobre nuestro servicio de envío.\nTus envíos\n¿Qué tipos de servicios ofrece Shalom?'),
 Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='¿Qué tipos de servicios ofrece Shalom?\nEn Shalom, nos especi

Una vez tengamos nuestros documentos cargados y partidos, podemos almacenarlos en el Vector Store para poder recuperarlos en tiempo real.

Dado que los campos son almacenados como vectores, necesitaremos transformar los textos usando un modelo de embeddings.

Para este ejemplo usaremos el modelo de embeddings de Nomic el cual nos da 1 Millon de tokens gratis en su plan gratuito. Ver: https://atlas.nomic.ai/

Una lista completa de los modelos de embeddings disponibles en LangChain pueden ser encontrados en https://python.langchain.com/v0.1/docs/integrations/text_embedding/

In [16]:
%pip install -qU langchain-nomic

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_nomic import NomicEmbeddings

embeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5"
)

embeddings.embed_query("Hola Python!!!")

[-0.029571533,
 0.047302246,
 -0.14819336,
 -0.030380249,
 0.039245605,
 0.008026123,
 -0.04864502,
 0.025314331,
 -0.014022827,
 -0.013069153,
 0.012557983,
 0.08618164,
 0.017486572,
 -0.025253296,
 0.024734497,
 -0.07623291,
 0.06311035,
 -0.013694763,
 -0.057037354,
 0.074645996,
 0.0006632805,
 -0.02645874,
 -0.046295166,
 -0.020614624,
 0.11254883,
 -0.030197144,
 -0.029388428,
 -0.0010070801,
 0.03918457,
 0.061828613,
 -0.0013465881,
 -0.036102295,
 -0.035736084,
 -0.04397583,
 -0.033477783,
 -0.017929077,
 0.09259033,
 -0.011734009,
 0.030715942,
 0.059631348,
 -0.061401367,
 0.007209778,
 -0.018936157,
 -0.016555786,
 0.029693604,
 -0.012565613,
 0.09399414,
 -0.03302002,
 -0.0045700073,
 0.009689331,
 -0.004436493,
 0.012367249,
 -0.05697632,
 -0.024169922,
 -0.00073337555,
 0.062927246,
 0.066223145,
 -0.01789856,
 0.044158936,
 0.018035889,
 0.05456543,
 0.061828613,
 -0.08514404,
 0.083862305,
 0.040008545,
 -0.043548584,
 -0.009277344,
 0.04736328,
 -0.058563232,
 0.0125

Almacenaremos los embeddings procesados en Chroma para poder recuperarlos más adelante.

Pueden revisar la documentación de Chroma en https://www.trychroma.com/ y la integración con LangChain en https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [18]:
from langchain_chroma import Chroma
from langchain_nomic import NomicEmbeddings

embedding = NomicEmbeddings(
    model="nomic-embed-text-v1.5"
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

vectorstore

El Retriever nos permitirá recuperar la información almacenada en nuestro Vector Store.

Por debajo trasnforma la pregunta del usuario en un vector y busca los vectores más cercanos en la base de datos vectorial.

In [19]:
retriever = vectorstore.as_retriever(k=5)
docs = retriever.invoke("Que medios de pago tienen?")

docs

[Document(metadata={'language': 'es', 'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom'}, page_content='¿Cuáles son los medios para realizar el pago de mi envío?\nShalom pone a su disposición nuestras 3 plataformas donde puede realizar el pago de sus envíos.'),
 Document(metadata={'language': 'es', 'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom'}, page_content='Página web: https://pagalo.shalom.pe\nApp Shalom: https://apple.co/3YNs2AK / https://cutt.ly/bXjJyz\nShalom Pro: https://pro.shalom.pe\n\nRecuerda que puedes realizar tus pagos con cualquiera de estos métodos:\n\nBilletera digital: Yape, Plin, Lukita, Tunki\nTarjeta\nEfectivo *En cualquiera de nuestras oficinas*\nPagos en Agencia, recuerd

### Chatbot con RAG

Finalmente, usaremos el modelo de RAG para recuperar la información relevante de la base de datos vectorial y agregarla al Prompt del Chatbot.

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain

load_dotenv(override=True) 

llm = ChatGroq(
    temperature=1.0,
    model="llama-3.1-8b-instant",
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente virtual. Tu nombre es {assistant_name} y trabajas en {company}. {description}. Usa las siguiente referencias como guía para tus respuestas: {context}"),
    MessagesPlaceholder(variable_name="messages"),
])

chat_history = ChatMessageHistory()

retriever = vectorstore.as_retriever(k=5)


In [21]:
def send_question(question):
    chat_history.add_user_message(question)
    docs = retriever.invoke(question)

    chain = create_stuff_documents_chain(llm, prompt)

    ai_response = chain.invoke({
        "assistant_name": "Azzy",
        "company": "Shalom",
        "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
        "messages": chat_history.messages,
        "context": docs
    })

    chat_history.add_ai_message(ai_response)
    return chat_history.messages

question = "Hola, ¿Que debo presentar para recoger un envio?"
send_question(question)

[HumanMessage(content='Hola, ¿Que debo presentar para recoger un envio?'),
 AIMessage(content='Hola, para recoger un envío con Shalom, debes presentar:\n\n- Si eres persona natural: Tu DNI original y la clave de seguridad.\n- Si eres empresa y tienes el contacto registrado en el ticket OS Shalom: Tu DNI original y la clave de seguridad.\n- Si eres empresa y no tienes el contacto registrado en el ticket OS Shalom: Una carta membretada debidamente firmada por el representante legal, copia de DNI, tu DNI original y la digitación de la clave de seguridad por tu parte.\n\nRecuerda que es importante verificar que la información sea correcta para evitar cualquier problema. ¿Necesitas ayuda adicional?')]

In [22]:
send_question("Gracias ... y ¿que medios de pago tienen?")

[HumanMessage(content='Hola, ¿Que debo presentar para recoger un envio?'),
 AIMessage(content='Hola, para recoger un envío con Shalom, debes presentar:\n\n- Si eres persona natural: Tu DNI original y la clave de seguridad.\n- Si eres empresa y tienes el contacto registrado en el ticket OS Shalom: Tu DNI original y la clave de seguridad.\n- Si eres empresa y no tienes el contacto registrado en el ticket OS Shalom: Una carta membretada debidamente firmada por el representante legal, copia de DNI, tu DNI original y la digitación de la clave de seguridad por tu parte.\n\nRecuerda que es importante verificar que la información sea correcta para evitar cualquier problema. ¿Necesitas ayuda adicional?'),
 HumanMessage(content='Gracias ... y ¿que medios de pago tienen?'),
 AIMessage(content='Shalom tiene disponibles varias opciones para realizar el pago de tus envíos. Puedes hacerlo a través de:\n\n1. **Página web**: Puedes hacer el pago en línea en nuestra página web https://pagalo.shalom.pe\n

### Gracias

Espero que esta charla les haya sido de ayuda. Si tienen alguna pregunta, no duden en contactarme en LinkedIn: https://www.linkedin.com/in/willywg/

También pueden agregame en GitHub: https://github.com/willywg